In [11]:
import os
import numpy as np
import pandas as pd
import string
import urllib.request
import zipfile

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from autocorrect import spell
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize as wt 

nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data1 = pd.read_csv(r'/content/ACLED Nigeria 2019-2020.csv')
data2 = pd.read_csv(r'/content/ACLED Nigeria 2020-2022.csv')
df = pd.concat([data1, data2], axis=0)

In [3]:
df = df[['notes', 'source']]
label_encoder = preprocessing.LabelEncoder()
df['source']= label_encoder.fit_transform(df['source'])

In [4]:
data = []
for i in range(df.shape[0]):
    sms = df.iloc[i, 0]
    sms = re.sub('[^A-Za-z]', ' ', str(sms))

    # make words lowercase, because Go and go will be considered as two words
    sms = sms.lower()

    # tokenising
    tokenized_sms = wt(sms)

    # remove stop words and stemming
 
    sms_processed = []
    for word in tokenized_sms:
      if word not in set(stopwords.words('english')):
        sms_processed.append(stemmer.stem(word))

    sms_text = " ".join(sms_processed)
    data.append(sms_text)

In [5]:
# creating the feature matrix 
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=1000)
X = matrix.fit_transform(data).toarray()
y = df.iloc[:, 1]

# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# predict class
y_pred = classifier.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

In [10]:
accuracy

0.11248220218319886